<a href="https://colab.research.google.com/github/sudikshakalepu/sentiment-analysis/blob/main/sentiment_analysis_lstm_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

import spacy

import nltk
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

from functools import reduce

In [53]:
path1 = "/content/sample_data/restaurantReviews10k.csv"
df = pd.read_csv(path1)

df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [54]:
df = df[['Review', 'Rating']]
df = df.drop_duplicates()
df['Review'] = df['Review'].fillna("")
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['Positive/Negative'] = df['Rating'].apply(lambda x: 0 if x < 3 else 1)
df.shape
df

<ipython-input-54-8c3a4da6098e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review'] = df['Review'].fillna("")
<ipython-input-54-8c3a4da6098e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
<ipython-input-54-8c3a4da6098e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,Review,Rating,Positive/Negative
0,"The ambience was good, food was quite good . h...",5.0,1
1,Ambience is too good for a pleasant evening. S...,5.0,1
2,A must try.. great food great ambience. Thnx f...,5.0,1
3,Soumen das and Arun was a great guy. Only beca...,5.0,1
4,Food is good.we ordered Kodi drumsticks and ba...,5.0,1
...,...,...,...
9995,Madhumathi Mahajan Well to start with nice cou...,3.0,1
9996,This place has never disappointed us.. The foo...,4.5,1
9997,"Bad rating is mainly because of ""Chicken Bone ...",1.5,0
9998,I personally love and prefer Chinese Food. Had...,4.0,1


In [55]:
nltk.download('stopwords')
nltk.download('punkt')

df = df.reset_index(drop=True)

df['reduced'] = ""
stop_words = set(stopwords.words('english'))

def clean_text(review):
    if not isinstance(review, str):
        return ""

    tokens = nltk.word_tokenize(review)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(filtered_tokens)

df['reduced'] = df['Review'].apply(clean_text)

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Review,Rating,Positive/Negative,reduced
0,"The ambience was good, food was quite good . h...",5.0,1,ambience good food quite good Saturday lunch c...
1,Ambience is too good for a pleasant evening. S...,5.0,1,Ambience good pleasant evening Service prompt ...
2,A must try.. great food great ambience. Thnx f...,5.0,1,must try great food great ambience Thnx servic...
3,Soumen das and Arun was a great guy. Only beca...,5.0,1,Soumen das Arun great guy behavior sincerety g...
4,Food is good.we ordered Kodi drumsticks and ba...,5.0,1,Food ordered Kodi drumsticks basket mutton bir...


In [56]:
# PREPROCESSING: perform lemmatization
nlp = spacy.load('en_core_web_sm')

df = df.reset_index(drop=True)

df['lemmatized'] = ""

for index, row in df.iterrows():
    temp = row['reduced']
    doc = nlp(temp)
    tokens = []
    for token in doc:
      tokens.append(token)
    lemma = " ".join([token.lemma_ for token in doc])

    df.at[index,'lemmatized'] = lemma

df.head()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


,Review,Rating,Positive/Negative,reduced,lemmatized
0,"The ambience was good, food was quite good . h...",5.0,1,ambience good food quite good Saturday lunch c...,ambience good food quite good Saturday lunch c...
1,Ambience is too good for a pleasant evening. S...,5.0,1,Ambience good pleasant evening Service prompt ...,ambience good pleasant evening Service prompt ...
2,A must try.. great food great ambience. Thnx f...,5.0,1,must try great food great ambience Thnx servic...,must try great food great ambience Thnx servic...
3,Soumen das and Arun was a great guy. Only beca...,5.0,1,Soumen das Arun great guy behavior sincerety g...,Soumen das Arun great guy behavior sincerety g...
4,Food is good.we ordered Kodi drumsticks and ba...,5.0,1,Food ordered Kodi drumsticks basket mutton bir...,food order Kodi drumstick basket mutton biryan...


In [57]:
# CHECK CLASS DISTRIBUTION
df['Positive/Negative'].value_counts()

1    6987
0    2422
Name: Positive/Negative, dtype: int64

In [58]:
# SEPARATE FEATURES AND TARGET VARIABLES
X_raw = df["Review"]
y_raw = df["Positive/Negative"]

X_prep = df["lemmatized"]
y_prep = df["Positive/Negative"].copy()

In [59]:
# TRAIN-TEST SPLIT 75-10-15 SCHEME.
XR_train, XR_test , yR_train, yR_test = train_test_split(X_raw,
                                                     y_raw,
                                                     test_size=0.15,
                                                     random_state=0,
                                                     stratify=y_raw)

XR_train, XR_val , yR_train, yR_val = train_test_split(XR_train,
                                                     yR_train,
                                                     test_size=0.10,
                                                     random_state=0,
                                                     stratify=yR_train)

XP_train, XP_test , yP_train, yP_test = train_test_split(X_prep,
                                                     y_prep,
                                                     test_size=0.15,
                                                     random_state=0,
                                                     stratify=y_prep)

XP_train, XP_val , yP_train, yP_val = train_test_split(XP_train,
                                                     yP_train,
                                                     test_size=0.10,
                                                     random_state=0,
                                                     stratify=yP_train)

XR_train = XR_train.fillna("")
XR_val = XR_val.fillna("")
XR_test = XR_test.fillna("")

XP_train = XP_train.fillna("")
XP_val = XP_val.fillna("")
XP_test = XP_test.fillna("")

In [60]:
# CREATE DENSE VECTORS: 100d GloVe trained on Twitter data

def createVocabulary_Glove(dataframe):
  vocab = []

  for i in dataframe:
    if not isinstance(i, str): continue
    tokens = [token.lower() for token in word_tokenize(i)]
    vocab.append(tokens)
  return vocab

raw_corpus_glove = createVocabulary_Glove(X_raw)
prep_corpus_glove = createVocabulary_Glove(X_prep)

raw_corpus_len = reduce(lambda count, l: count + len(l), raw_corpus_glove, 0)
prep_corpus_len = reduce(lambda count, l: count + len(l), prep_corpus_glove, 0)

print("Raw corpus length: ", raw_corpus_len)
print("Prepped corpus length: ", prep_corpus_len)

Raw corpus length:  563669
Prepped corpus length:  266962


In [61]:
# PAD DATASET TO MATCH THE GLOVE EMBEDDING SIZE

embedding_size = 100
tokenizer_R = Tokenizer(num_words = raw_corpus_len)

tokenizer_R.fit_on_texts(XR_train)
pickle.dump(tokenizer_R, open('glove_tokenizer.pkl', 'wb'))

XR_train_seq = tokenizer_R.texts_to_sequences(XR_train)
XR_train_pad = pad_sequences(XR_train_seq, maxlen = embedding_size, truncating = "post", padding = "post")
XR_val_seq = tokenizer_R.texts_to_sequences(XR_val)
XR_val_pad = pad_sequences(XR_val_seq, maxlen = embedding_size, truncating = "post", padding = "post")
XR_test_seq = tokenizer_R.texts_to_sequences(XR_test)
XR_test_pad = pad_sequences(XR_test_seq, maxlen = embedding_size, truncating = "post", padding = "post")

tokenizer_P = Tokenizer(num_words = prep_corpus_len)
tokenizer_P.fit_on_texts(XP_train)

XP_train_seq = tokenizer_P.texts_to_sequences(XP_train)
XP_train_pad = pad_sequences(XP_train_seq, maxlen = embedding_size, truncating = "post", padding = "post")
XP_val_seq = tokenizer_R.texts_to_sequences(XP_val)
XP_val_pad = pad_sequences(XP_val_seq, maxlen = embedding_size, truncating = "post", padding = "post")
XP_test_seq = tokenizer_P.texts_to_sequences(XP_test)
XP_test_pad = pad_sequences(XP_test_seq, maxlen = embedding_size, truncating = "post", padding = "post")

In [62]:
# CREATE WORD INDEX FROM TOKENIZER

raw_word_index = tokenizer_R.word_index
prep_word_index = tokenizer_P.word_index

In [63]:
# CREATE GLOVE EMBEDDING MATRIX

glove_vectors = {}
with open("/content/sample_data/glove.twitter.27B.100d.txt", "r") as f:
  for line in f:
    x = line.split()
    word = x[0]
    vector = np.asarray(x[1:], "float32")
    glove_vectors[word] = vector
f.close()

In [64]:
# GENERATE WORD REPRESENTATION OF DATASET USING GLOVE EMBEDDING MATRIX

n_r = len(raw_word_index) + 1
n_p = len(prep_word_index) + 1

rep_matrix_raw = np.zeros((n_r, 100))
rep_matrix_prep = np.zeros((n_p, 100))

for w, i in raw_word_index.items():
  word_rep = glove_vectors.get(w)
  if i < n_r:
    if word_rep is not None:
      rep_matrix_raw[i] = word_rep

for w, i in prep_word_index.items():
  word_rep = glove_vectors.get(w)
  if i < n_r:
    if word_rep is not None:
      rep_matrix_prep[i] = word_rep

In [65]:
# CHECK IF WORD REPRESENTATIONS MADE PROPERLY
raw_word_index["good"]

9

In [66]:
# CHECK IF WORD REPRESENTATIONS MADE PROPERLY
glove_vectors.get("good")

array([ 9.1552e-02,  9.3336e-02, -2.8113e-02,  3.6990e-01,  1.8956e-01,
        4.3191e-01,  1.0266e-01,  3.4092e-01, -3.8717e-01,  5.0772e-02,
       -4.8289e-01, -3.7883e-01, -5.0020e+00, -2.7927e-01, -1.4578e-01,
       -1.0818e-01,  3.9178e-01, -4.5798e-01, -5.4712e-01, -2.0868e-01,
       -3.3495e-02,  1.0765e-01, -1.7093e-01, -1.0107e-01,  1.1071e-01,
       -1.7558e-01, -3.5739e-01,  1.5897e-01,  7.9770e-02, -9.7594e-01,
       -3.9054e-01,  1.5097e-01, -5.6339e-01,  5.0047e-01,  1.7436e-01,
        6.8966e-01,  2.6879e-01, -4.7430e-01, -1.4947e-02,  2.3483e-01,
       -9.4609e-01, -5.5384e-02,  2.7673e-01,  4.1185e-01,  3.5373e-01,
        3.6709e-01, -1.4923e-01, -6.6414e-02,  2.5766e-01, -3.6882e-03,
       -6.6522e-01,  7.5965e-02, -3.0112e-01, -1.5115e-01,  3.5540e-01,
        5.9782e-01, -2.0585e-01, -4.8643e-01,  6.6179e-01, -1.3902e-01,
        2.4042e-02, -9.1297e-01, -2.1795e-01,  4.9977e-01,  4.9224e-02,
       -4.8337e-01, -5.1529e-01, -6.8428e-01, -2.4887e-01,  5.98

In [67]:
# CHECK IF WORD REPRESENTATIONS MADE PROPERLY
(rep_matrix_raw[21] == glove_vectors.get("good")).all()

False

In [68]:
# TRAIN MODELS: LSTM with dense representations of raw data, no dropout

lstm_dense_raw = Sequential()

lstm_dense_raw.add(Embedding(n_r, embedding_size, embeddings_initializer = Constant(rep_matrix_raw), input_length = embedding_size, trainable = False))
lstm_dense_raw.add(LSTM(embedding_size, return_sequences = True))
lstm_dense_raw.add(LSTM(embedding_size))
lstm_dense_raw.add(Dense(1, activation = "sigmoid"))
optimizer = Adam(learning_rate = 3e-4)
lstm_dense_raw.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

print(lstm_dense_raw.summary())

history = lstm_dense_raw.fit(XR_train_pad, yR_train, epochs = 20, validation_data = (XR_val_pad, yR_val), verbose = 1)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 100, 100)          1425400   
                                                                 
 lstm_12 (LSTM)              (None, 100, 100)          80400     
                                                                 
 lstm_13 (LSTM)              (None, 100)               80400     
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1586301 (6.05 MB)
Trainable params: 160901 (628.52 KB)
Non-trainable params: 1425400 (5.44 MB)
_________________________________________________________________
None
Epoch 1/20
225/225 [==============================] - 7s 17ms/step - loss: 0.5688 - accuracy: 0.7477 - val_loss: 0.5681 - val_accuracy: 0

In [69]:
# CHECK ACCURACY: LSTM with dense representations of raw data, no dropout

s1 = lstm_dense_raw.evaluate(XR_test_pad, yR_test, verbose=0)
print("Accuracy: %.2f%%" % (s1[1]*100))

Accuracy: 89.66%


In [70]:
# TRAIN MODELS: LSTM with dense representations of processed data, no dropout

lstm_dense_prep = Sequential()

lstm_dense_prep.add(Embedding(n_p, embedding_size, embeddings_initializer = Constant(rep_matrix_prep), input_length = embedding_size, trainable = False))
lstm_dense_prep.add(LSTM(embedding_size, return_sequences = True))
lstm_dense_prep.add(LSTM(embedding_size))
lstm_dense_prep.add(Dense(1, activation = "sigmoid"))
optimizer = Adam(learning_rate = 3e-4)
lstm_dense_prep.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

print(lstm_dense_prep.summary())

history = lstm_dense_prep.fit(XP_train_pad, yP_train, epochs = 20, validation_data = (XP_val_pad, yP_val), verbose = 1)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 100)          1089900   
                                                                 
 lstm_14 (LSTM)              (None, 100, 100)          80400     
                                                                 
 lstm_15 (LSTM)              (None, 100)               80400     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1250801 (4.77 MB)
Trainable params: 160901 (628.52 KB)
Non-trainable params: 1089900 (4.16 MB)
_________________________________________________________________
None
Epoch 1/20
225/225 [==============================] - 7s 15ms/step - loss: 0.5323 - accuracy: 0.7460 - val_loss: 0.6637 - val_accuracy: 0

In [71]:
# SET INPUT SIZES FOR MODEL TRAINING

num_words_raw = raw_corpus_len + 1
num_words_prep = prep_corpus_len + 1

raw_tf_embedding_dim = XR_train_tf.shape[1]
prep_tf_embedding_dim = XP_train_tf.shape[1]

In [72]:
# TRAIN MODEL: LSTM with dense representations of raw data, with dropout

lstm_dense_raw_drop = Sequential()

lstm_dense_raw_drop.add(Embedding(n_r, embedding_size, embeddings_initializer = Constant(rep_matrix_raw), input_length = embedding_size, trainable = False))
lstm_dense_raw_drop.add(LSTM(embedding_size, return_sequences = True))
lstm_dense_raw_drop.add(Dropout(0.1))
lstm_dense_raw_drop.add(LSTM(embedding_size))
lstm_dense_raw_drop.add(Dense(1, activation = "sigmoid"))
optimizer = Adam(learning_rate = 3e-4)
lstm_dense_raw_drop.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

print(lstm_dense_raw_drop.summary())

history = lstm_dense_raw_drop.fit(XR_train_pad, yR_train, epochs = 20, validation_data = (XR_val_pad, yR_val), verbose = 1)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 100, 100)          1425400   
                                                                 
 lstm_16 (LSTM)              (None, 100, 100)          80400     
                                                                 
 dropout_4 (Dropout)         (None, 100, 100)          0         
                                                                 
 lstm_17 (LSTM)              (None, 100)               80400     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1586301 (6.05 MB)
Trainable params: 160901 (628.52 KB)
Non-trainable params: 1425400 (5.44 MB)
_________________________________________________________________
None
Epoch

In [73]:
# TRAIN MODEL: LSTM with dense representations of processed data, with dropout

lstm_dense_prep_drop = Sequential()

lstm_dense_prep_drop.add(Embedding(n_p, embedding_size, embeddings_initializer = Constant(rep_matrix_prep), input_length = embedding_size, trainable = False))
lstm_dense_prep_drop.add(LSTM(embedding_size, return_sequences = True))
lstm_dense_prep_drop.add(Dropout(0.1))
lstm_dense_prep_drop.add(LSTM(embedding_size))
lstm_dense_prep_drop.add(Dense(1, activation = "sigmoid"))
optimizer = Adam(learning_rate = 3e-4)
lstm_dense_prep_drop.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

print(lstm_dense_prep_drop.summary())

history = lstm_dense_prep_drop.fit(XP_train_pad, yP_train, epochs = 20, validation_data = (XP_val_pad, yP_val), verbose = 1)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 100, 100)          1089900   
                                                                 
 lstm_18 (LSTM)              (None, 100, 100)          80400     
                                                                 
 dropout_5 (Dropout)         (None, 100, 100)          0         
                                                                 
 lstm_19 (LSTM)              (None, 100)               80400     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1250801 (4.77 MB)
Trainable params: 160901 (628.52 KB)
Non-trainable params: 1089900 (4.16 MB)
_________________________________________________________________
None
Epoch

In [79]:
# CHECK ACCURACY: all LSTM models
print("Accuracy of LSTM with GloVe embedding \non raw/processed data, with/without dropout")
accuracy_values = {'Without Dropout': [s1[1]*100, s2[1]*100], 'With Dropout': [s3[1]*100, s4[1]*100]}
data_types = ['Raw Data', 'Processed Data']
df = pd.DataFrame(accuracy_values, index=data_types)
df

Accuracy of LSTM with GloVe embedding 
on raw/processed data, with/without dropout


,Without Dropout,With Dropout
Raw Data,89.660054,91.713881
Processed Data,74.150139,68.342775
